# Lab 1

All code below is referenced from Lecture_1_2.ipynb provided by Gittu George for DSCI 525

In [1]:
# import packages

import dask.dataframe as dd
import re
import os
import glob
import sys
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
from memory_profiler import memory_usage
from os import listdir
from functools import reduce

In [2]:
#%load_ext rpy2.ipython

In [3]:
%load_ext memory_profiler

In [4]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [5]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

Next, we download the data:

In [6]:
# make directory if missing
os.makedirs(output_directory, exist_ok=True)

# download missing files
files_to_dl = ["data.zip"]
for item in filter(lambda x: x['name'] in files_to_dl, files):
    filename = os.path.join(output_directory, item["name"])
    if not os.path.isfile(filename):
        urlretrieve(item["download_url"], filename)

The download took 55 mins on Raf's computer.

## Method 1: Pandas from ZIP Directly

In [7]:
# open a read-only connection to zip file
zfile = zipfile.ZipFile(glob.glob(output_directory + "*.zip")[0], "r")

# list non-hidden files in zip
z_csvs = list(filter(lambda x: not x.startswith("__"), zfile.namelist()))
z_csvs = [x for x in z_csvs if "observed" not in x]

In [8]:
%%time
%%memit

# create a dictionary of dataframes
dat = {x.split('_daily')[0]: pd.read_csv(zfile.open(x)) for x in z_csvs}

# reshape to one big dataframe
dat = pd.concat(dat, names=["model", "row"])

peak memory: 10995.75 MiB, increment: 10895.57 MiB
CPU times: user 1min 2s, sys: 5.43 s, total: 1min 7s
Wall time: 1min 8s


Unfortunately, this method is very memory hungry. It requires 10,361 MB of memory.

In [9]:
print(dat.shape)

(62467843, 6)


In [10]:
dat.head()

time    lat_min    lat_max   lon_min  \
model           row                                                        
MPI-ESM-1-2-HAM 0    1889-01-01 12:00:00 -35.439867 -33.574619  141.5625   
                1    1889-01-02 12:00:00 -35.439867 -33.574619  141.5625   
                2    1889-01-03 12:00:00 -35.439867 -33.574619  141.5625   
                3    1889-01-04 12:00:00 -35.439867 -33.574619  141.5625   
                4    1889-01-05 12:00:00 -35.439867 -33.574619  141.5625   

                      lon_max  rain (mm/day)  
model           row                           
MPI-ESM-1-2-HAM 0    143.4375   4.244226e-13  
                1    143.4375   4.217326e-13  
                2    143.4375   4.498125e-13  
                3    143.4375   4.251282e-13  
                4    143.4375   4.270161e-13

## Method 2: Using Pandas on Unzipped Files

In [11]:
del dat

In [12]:
%%time

# Unzip the files
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 16.4 s, sys: 2.19 s, total: 18.6 s
Wall time: 19.8 s


In [13]:
# list all CSVs
csvs = glob.glob(output_directory + '*.csv')


# As per Tom's guidance, we can exclude the annoying CSV that is formatted differently
csvs = [x for x in csvs if "observed" not in x]

In [14]:
%%time
%%memit

# create a dictionary of dataframes
dat = {x.split('_daily')[0]: pd.read_csv(x) for x in csvs}

# reshape to one big dataframe
dat = pd.concat(dat, names=["model", "row"])

peak memory: 15030.56 MiB, increment: 6200.90 MiB
CPU times: user 51.8 s, sys: 7 s, total: 58.8 s
Wall time: 59.9 s


In [15]:
assert dat.shape[0] == 62467843

Looking at the incremental columns, it seems that reduced the memory requirement by ~40% by using unzipped files.

## Method 3: Append, instead of Using pd.concat

In [16]:
del dat

In [17]:
%%time
%%memit

dat = pd.DataFrame()
for file in csvs:
    tempdf = pd.read_csv(file)
    tempdf['model'] = file.split('_daily')[0]
    dat = dat.append(tempdf)

peak memory: 14990.10 MiB, increment: 6441.43 MiB
CPU times: user 2min, sys: 27.4 s, total: 2min 27s
Wall time: 2min 29s


In [18]:
dat.shape[0]

62467843

In [19]:
assert dat.shape[0] == 62467843

There is no appreciable difference in memory usage, but it is much slower than `pd.concat`.

### Method 4: Change the data type

In [20]:
# delete dat object to read it in anew
del dat

In [21]:
# define the dtypes
colspec = {"time": "str",
           "lat_min": np.float32,
           "lat_max": np.float32,
           "lon_min": np.float32,
           "lon_max": np.float32,
           "rain (mm/day)": np.float32}

In [22]:
%%time
%%memit

# create a dictionary of dataframes
dat = {x.split('_daily')[0]: pd.read_csv(x, dtype=colspec, parse_dates=["time"]) for x in csvs}

# reshape to one big dataframe
dat = pd.concat(dat, names=["model", "row"])

peak memory: 11515.95 MiB, increment: 4109.92 MiB
CPU times: user 55.1 s, sys: 5.1 s, total: 1min
Wall time: 1min 1s


In [23]:
dat.head()

time    lat_min    lat_max  \
model                           row                                             
figshareairline/MPI-ESM-1-2-HAM 0   1889-01-01 12:00:00 -35.439865 -33.574619   
                                1   1889-01-02 12:00:00 -35.439865 -33.574619   
                                2   1889-01-03 12:00:00 -35.439865 -33.574619   
                                3   1889-01-04 12:00:00 -35.439865 -33.574619   
                                4   1889-01-05 12:00:00 -35.439865 -33.574619   

                                      lon_min   lon_max  rain (mm/day)  
model                           row                                     
figshareairline/MPI-ESM-1-2-HAM 0    141.5625  143.4375   4.244226e-13  
                                1    141.5625  143.4375   4.217326e-13  
                                2    141.5625  143.4375   4.498125e-13  
                                3    141.5625  143.4375   4.251282e-13  
                                4    141.5625  143.4375   4.270161e-13

Changing the memory type to float 32 gives us a ~30% boost in our memory footprint. Timing is about the same.

## Method 5: Only Load Wanted Columns

In [24]:
del dat

In [25]:
usecols = ["time", "rain (mm/day)"]

In [26]:
%%time
%%memit

# create a dictionary of dataframes
dat = {x.split('_daily')[0]: pd.read_csv(x, dtype=colspec, parse_dates=["time"], usecols=usecols) for x in csvs}

# reshape to one big dataframe
dat = pd.concat(dat, names=["model", "row"])

peak memory: 10441.61 MiB, increment: 2891.75 MiB
CPU times: user 45.2 s, sys: 4.21 s, total: 49.4 s
Wall time: 50.3 s


In [27]:
dat.head()

time  rain (mm/day)
model                           row                                   
figshareairline/MPI-ESM-1-2-HAM 0   1889-01-01 12:00:00   4.244226e-13
                                1   1889-01-02 12:00:00   4.217326e-13
                                2   1889-01-03 12:00:00   4.498125e-13
                                3   1889-01-04 12:00:00   4.251282e-13
                                4   1889-01-05 12:00:00   4.270161e-13

This step made a big difference, cutting down the memory usage by about a quarter. It gave us a small speed advantage too.

## Method 6: Chunking

In [28]:
del dat

In [29]:
%%time
%%memit

dat = pd.DataFrame()
for file in csvs:
    model = file.split('_daily')[0]
    for chunk in pd.read_csv(file, dtype=colspec, parse_dates=["time"], usecols=usecols,
                             chunksize=10_000_000):
        chunk['model'] = model
        dat = pd.concat([dat, chunk])

peak memory: 13472.99 MiB, increment: 5409.55 MiB
CPU times: user 53.7 s, sys: 12 s, total: 1min 5s
Wall time: 1min 6s


In [30]:
assert dat.shape[0] == 62467843

Let's try even smaller chunks:

In [31]:
del dat

In [32]:
%%time
%%memit

dat = pd.DataFrame()
for file in csvs:
    model = file.split('_daily')[0]
    for chunk in pd.read_csv(file, dtype=colspec, parse_dates=["time"], usecols=usecols,
                             chunksize=1_000_000):
        chunk['model'] = model
        dat = pd.concat([dat, chunk])

peak memory: 13554.91 MiB, increment: 225.83 MiB
CPU times: user 1min 8s, sys: 14.6 s, total: 1min 23s
Wall time: 1min 24s


In [33]:
assert dat.shape[0] == 62467843

These incremental memory results are phenomenal for chunk size of a million-- but almost too good to be true. I am not sure if I trust them completely.

## Method 7: Using Dask

In [34]:
del dat

In [35]:
%%time
%%memit

# define a parser to extract the model name
def parser(path):
    file = os.path.split(path)[1]
    return file.split('_daily')[0]

# read-in with dask
dat = dd.read_csv(csvs, include_path_column = "model",
                  converters = {"model": parser})

peak memory: 13333.35 MiB, increment: 0.46 MiB
CPU times: user 70 ms, sys: 119 ms, total: 189 ms
Wall time: 1.49 s


From the profiling, it is apparent that Dask has a negligible memory footprint.

In [36]:
dat.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [37]:
%%time
%%memit

len(dat.index)

peak memory: 13485.93 MiB, increment: 0.00 MiB
CPU times: user 1min 16s, sys: 16.2 s, total: 1min 32s
Wall time: 27.6 s


We can see that a 'simple' operation of counting the number of rows took a very long time!

In [38]:
%%time
%%memit

res = (dat
       .groupby('model')['rain (mm/day)']
       .agg(["mean", "std"])
       .sort_values("mean")
       .compute()
      )
print(res)

                      mean       std
model                               
MPI-ESM1-2-HR     0.995569  4.083814
MPI-ESM1-2-LR     1.074308  3.911700
KIOST-ESM         1.102353  3.852051
MRI-ESM2-0        1.368030  4.517987
GFDL-CM4          1.414485  5.024926
EC-Earth3-Veg-LR  1.516258  4.714335
MPI-ESM-1-2-HAM   1.610720  4.885519
NESM3             1.621936  4.971972
FGOALS-f3-L       1.627373  5.747396
ACCESS-CM2        1.787025  5.914188
BCC-ESM1          1.811032  5.358361
CanESM5           1.894328  5.835787
BCC-CSM2-MR       1.951832  6.200969
AWI-ESM-1-1-LR    2.026071  5.321889
FGOALS-g3         2.156419  6.015488
SAM0-UNICON       2.169676  6.383241
ACCESS-ESM1-5     2.217501  6.422397
TaiESM1           2.224576  5.886578
NorESM2-LM        2.230799  5.681562
NorESM2-MM        2.232966  6.151688
CMCC-ESM2         2.266125  5.538429
CMCC-CM2-HR4      2.279350  5.629965
MIROC6            2.301662  6.393745
CMCC-CM2-SR5      2.383389  5.895950
INM-CM5-0         2.669012  6.534084
I

Calculating model-wise summarise also took a long time-- almost a minute!

In [39]:
# import altair as alt

# alt.Chart(res, title = "Average Rainfall by Model").mark_point().encode(
#     alt.Y("model:Q"),
#     alt.X("mean:Q")
# )